In [1]:
# Install required packages (uncomment if needed)
# %pip install plotly ipywidgets

import copy
import torch
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display, clear_output
import time

# Use a notebook-friendly renderer for inline display
pio.renderers.default = 'notebook_connected'

In [2]:
# Optimizer classes (kept identical to visualizer.py)
class ADAM_Optimization: 
    def __init__(self, lr=0.3):
        self.B1 = 0.6
        self.B2 = 0.99
        self.eps = 1e-8
        self.lr = lr
        self.reset()

    def reset(self):
        self.t =0
        self.m = 0
        self.v= 0
    
    def update(self, grad):
        self.t += 1
        self.m = self.B1*self.m + (1-self.B1)*grad
        self.v = self.B2*self.v + (1-self.B2)*(grad**2)

        m_hat = self.m/(1-self.B1**self.t)
        v_hat = self.v/(1-self.B2**self.t)

        update = self.lr * m_hat/(v_hat+self.eps)**.5
        return update
class Adam_No_Correction(ADAM_Optimization):
    def __init__(self, lr=0.3):
        super().__init__(lr=lr)

    def update(self, grad):
        self.t += 1
        self.m = self.B1*self.m + (1-self.B1)*grad
        self.v = self.B2*self.v + (1-self.B2)*(grad**2)

        update = self.lr * self.m/(self.v+self.eps)**.5
        return update
class SGD:
    def __init__(self, lr=0.1):
        self.lr = lr
    
    def reset(self):
        pass

    def update(self, grad):
        return self.lr * grad
    
class RMSProp:
    def __init__(self, lr=0.1, beta=0.99, eps=1e-8):
        self.lr = lr
        self.beta = beta
        self.eps = eps
        self.reset()
    
    def reset(self):
        self.v = 0

    def update(self, grad):
        self.v = self.beta * self.v + (1-self.beta)*(grad**2)
        update = self.lr * grad/(self.v+self.eps)**.5
        return update

class RProp:
    def __init__(self, lr=0.1, eta_plus=1.2, eta_minus=0.5, delta_min=1e-6, delta_max=50.0):
        self.lr = lr
        self.eta_plus = eta_plus
        self.eta_minus = eta_minus
        self.delta_min = delta_min
        self.delta_max = delta_max
        self.reset()
    
    def reset(self):
        self.prev_grad = 0
        self.delta = 0.1  # Initial step size

    def update(self, grad):
        sign = grad * self.prev_grad
        update = np.sign(grad)
        if sign > 0:
            self.delta = min(self.delta * self.eta_plus, self.delta_max)
        elif sign < 0:
            self.delta = max(self.delta * self.eta_minus, self.delta_min)
        
        update *= self.delta
        
        self.prev_grad = grad
        return update
class SGD_Momentum:
    def __init__(self, lr=0.1, k_momentum=0.5):
        self.lr = lr
        self.k_momentum = k_momentum
        self.reset()
    
    def reset(self):
        self.v = 0

    def update(self, grad):
        self.v = self.k_momentum * self.v + self.lr * grad
        return self.v
class NAG:
    def __init__(self, lr=0.1, k_momentum=0.5):
        self.lr = lr
        self.k_momentum = k_momentum
        self.reset()
    
    def reset(self):
        self.v = 0

    def update(self, grad):
        lookahead_grad = grad + self.k_momentum * self.v
        self.v = self.k_momentum * self.v + self.lr * lookahead_grad
        return self.v
        
class Attention_Grad:
    def __init__(self, lr=0.3):
        self.reset()
        self.lr = lr
    
    def reset(self):
        self.direction = 0 
        self.momentum = 0
    def update(self, grad, direction):
        attention_score = (1-np.abs(direction - self.direction))
        self.momentum = (1-attention_score)*self.momentum + attention_score*grad
        self.direction = (1-attention_score)*self.direction + attention_score*direction
        return self.lr * self.momentum


In [3]:
# Surface class (same generation logic as visualizer.py)
class Surface:
    def __init__(self, num_peaks=50, x_range=(-20,20), y_range=(-10,10)):
        self.num_peaks = num_peaks
        self.x_range = x_range
        self.y_range = y_range
        self.x_centers = np.random.uniform(self.x_range[0], self.x_range[1], self.num_peaks)
        self.y_centers = np.random.uniform(self.y_range[0], self.y_range[1], self.num_peaks)
        self.x_widths = np.random.uniform(0.5, 2, self.num_peaks)
        self.y_widths = np.random.uniform(0.5, 2, self.num_peaks)

        self.x_centers = torch.tensor(self.x_centers, dtype=torch.float32)
        self.y_centers = torch.tensor(self.y_centers, dtype=torch.float32)
        self.x_widths = torch.tensor(self.x_widths, dtype=torch.float32)
        self.y_widths = torch.tensor(self.y_widths, dtype=torch.float32)
    
    def getSurface(self,x,y):
        z = (x**2 + y**2) / 100  # parabolic component
        for i in range(self.num_peaks):
            z += torch.exp(-( (x - self.x_centers[i])**2/(2*self.x_widths[i]**2) + (y - self.y_centers[i])**2 / (2 * self.y_widths[i]**2)))
        return z

In [4]:
# Simulation function (adapted for notebook)
def simulate_optimizer(optimizer, surface, no_of_frames=200, x_start=-19.0, y_start=-9.0):
    optx = copy.deepcopy(optimizer)
    optx.reset()
    opty = copy.deepcopy(optimizer)
    opty.reset()

    xs = [float(x_start)]
    ys = [float(y_start)]
    with torch.no_grad():
        z0 = surface.getSurface(torch.tensor([x_start]), torch.tensor([y_start]))
    zs = [float(z0.item())]

    for _ in range(no_of_frames):
        x_t = torch.tensor([xs[-1]], dtype=torch.float32, requires_grad=True)
        y_t = torch.tensor([ys[-1]], dtype=torch.float32, requires_grad=True)
        z = surface.getSurface(x_t,y_t)
        z.backward()
        dx = x_t.grad.item()
        dy = y_t.grad.item()

        upd_x = optx.update(dx)
        upd_y = opty.update(dy)

        new_x = xs[-1] - upd_x
        new_y = ys[-1] - upd_y
        new_z = surface.getSurface(torch.tensor([new_x]), torch.tensor([new_y]))

        xs.append(float(new_x))
        ys.append(float(new_y))
        zs.append(float(new_z.item()))
    return np.array(xs), np.array(ys), np.array(zs)


In [5]:
# Build Plotly figure (inline-friendly, coarser grid by default) with animation
# Per-frame x/y axis ranges computed from the optimizer traces for each frame
# Also accept x_start and y_start to pass starting point through to the simulator
def build_plotly_figure(surface, optimizers, names, colors, no_of_frames=200, grid_step=0.2,
                        x_start=-19.0, y_start=-9.0):
    # build meshgrid for surface
    np_X, np_Y = np.meshgrid(np.arange(surface.x_range[0], surface.x_range[1]+grid_step, grid_step),
                             np.arange(surface.y_range[0], surface.y_range[1]+grid_step, grid_step))
    X = torch.tensor(np_X, dtype=torch.float32)
    Y = torch.tensor(np_Y, dtype=torch.float32)
    Z = surface.getSurface(X, Y).numpy()

    # simulate each optimizer (returns arrays of length no_of_frames+1)
    sim_results = []
    for opt in optimizers:
        xs, ys, zs = simulate_optimizer(opt, surface, no_of_frames=no_of_frames, x_start=x_start, y_start=y_start)
        sim_results.append((xs, ys, zs))

    fig = go.Figure()

    # add surface (static) as the first trace
    surface_trace = go.Surface(x=np_X, y=np_Y, z=Z, colorscale='Plasma', opacity=0.6, showscale=False, name='surface')
    fig.add_trace(surface_trace)

    # Add initial traces for each optimizer: a line trace and a moving marker trace
    for i, (xs, ys, zs) in enumerate(sim_results):
        fig.add_trace(go.Scatter3d(x=[float(xs[0])], y=[float(ys[0])], z=[float(zs[0])],
                                   mode='lines+markers',
                                   line=dict(color=colors[i], width=4),
                                   marker=dict(size=3, color=colors[i]),
                                   name=names[i]))
        fig.add_trace(go.Scatter3d(x=[float(xs[0])], y=[float(ys[0])], z=[float(zs[0])],
                                   mode='markers',
                                   marker=dict(size=6, color=colors[i]),
                                   name=f"{names[i]}_marker",
                                   showlegend=False))

    # Build frames: update only the optimizer traces (leave surface static) by using Frame.traces
    frames = []
    n_frames = sim_results[0][0].shape[0]
    n_opts = len(sim_results)
    animated_trace_indices = []
    for i in range(n_opts):
        animated_trace_indices.extend([1 + 2*i, 1 + 2*i + 1])

    for t in range(n_frames):
        frame_data = []
        # build optimizer traces for this frame
        for i, (xs, ys, zs) in enumerate(sim_results):
            frame_data.append(go.Scatter3d(x=xs[:t+1].tolist(), y=ys[:t+1].tolist(), z=zs[:t+1].tolist(),
                                           mode='lines+markers',
                                           line=dict(color=colors[i], width=4),
                                           marker=dict(size=3, color=colors[i])))
            frame_data.append(go.Scatter3d(x=[float(xs[t])], y=[float(ys[t])], z=[float(zs[t])],
                                           mode='markers',
                                           marker=dict(size=8, color=colors[i])))

        # compute min/max for x and y at this frame across all traces (use current points only)
        all_x = np.concatenate([sr[0][:t+1] for sr in sim_results])
        all_y = np.concatenate([sr[1][:t+1] for sr in sim_results])
        x_min, x_max = float(all_x.min()), float(all_x.max())
        y_min, y_max = float(all_y.min()), float(all_y.max())

        span_x = x_max - x_min
        span_y = y_max - y_min
        max_span = max(span_x, span_y)

        # By default use padding; if x_radius/y_radius provided, use centered radius around current mean
        center_x = float(all_x.mean())
        center_y = float(all_y.mean())

        pad = max_span * 0.12 if max_span > 0 else 0.5
        x_range = [x_min - pad, x_max + pad]
        y_range = [y_min - pad, y_max + pad]

        # include camera and per-frame axis ranges in the frame layout
        frame_layout = go.Layout(scene=dict(xaxis=dict(range=x_range, autorange=False),
                                            yaxis=dict(range=y_range, autorange=False)))

        frames.append(go.Frame(data=frame_data, name=str(t), traces=animated_trace_indices, layout=frame_layout))

    fig.frames = frames

    # controls
    play_button = dict(
        type='buttons',
        showactive=False,
        y=1.05,
        x=0.8,
        xanchor='right',
        yanchor='top',
        pad=dict(t=0, r=10),
        buttons=[
            dict(label='Play', method='animate',
                 args=[None, dict(frame=dict(duration=50, redraw=True), fromcurrent=True, transition=dict(duration=0))]),
            dict(label='Pause', method='animate',
                 args=[[None], dict(frame=dict(duration=0, redraw=False), mode='immediate', transition=dict(duration=0))])
        ]
    )

    steps = []
    for k in range(n_frames):
        steps.append(dict(method='animate', args=[[str(k)], dict(mode='immediate', frame=dict(duration=0, redraw=True), transition=dict(duration=0))], label=str(k)))

    sliders = [dict(active=0, pad=dict(t=50), steps=steps, x=0.1, y=0, currentvalue=dict(font=dict(size=12), prefix='Frame: ', visible=True, xanchor='right'))]

    fig.update_layout(updatemenus=[play_button], sliders=sliders)

    # apply initial scene settings from frame 0 so first render matches
    fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'), width=900, height=700)
    return fig


In [ ]:
# Widgets and interactive controls
output = widgets.Output()

optimizer_select = widgets.SelectMultiple(options=['ADAM','SGD','RMSProp','SGD_Mom', 'RProp', "Adam_No_Correction", 'NAG'], value=['ADAM','SGD','RMSProp','SGD_Mom', 'RProp', 'Adam_No_Correction', 'NAG'], description='Optimizers')
frames_slider = widgets.IntSlider(value=200, min=20, max=500, step=10, description='Frames')
x_start = widgets.FloatSlider(value=-19.0, min=-20.0, max=20.0, step=0.5, description='x_start')
y_start = widgets.FloatSlider(value=-9.0, min=-10.0, max=10.0, step=0.5, description='y_start')
peak_count = widgets.IntSlider(value=50, min=1, max=200, step=1, description='Peaks')

run_button = widgets.Button(description='Run / Update')
save_button = widgets.Button(description='Save HTML')

controls = widgets.VBox([optimizer_select, frames_slider, x_start, y_start, peak_count, widgets.HBox([run_button, save_button])])

def build_optimizers_from_selection(selection, lr=0.1):
    opts = []
    for name in selection:
        if name == 'ADAM':
            opts.append(ADAM_Optimization(lr=lr))
        if name == 'Adam_No_Correction':
            opts.append(Adam_No_Correction(lr =lr))
        elif name == 'SGD':
            opts.append(SGD(lr=lr))
        elif name == 'RMSProp':
            opts.append(RMSProp(lr=lr))
        elif name == 'SGD_Mom':
            opts.append(SGD_Momentum(lr=lr, k_momentum=0.9))
        elif name == 'RProp':
            opts.append(RProp(lr=lr))
        elif name == 'NAG':
            opts.append(NAG(lr=lr))
    return opts

current_fig = None

def on_run_clicked(b):
    global current_fig
    with output:
        clear_output(wait=True)
        surf = Surface(num_peaks=peak_count.value)
        names = list(optimizer_select.value)
        optimizers = build_optimizers_from_selection(names)
        colors = ['black','red','green','blue','orange','purple', 'yellow', 'magenta'][:len(optimizers)]
        fig = build_plotly_figure(surf, optimizers, names, colors, no_of_frames=frames_slider.value, grid_step=0.2,
                                 x_start=x_start.value, y_start=y_start.value)
        current_fig = fig
        fig.show()

def on_save_clicked(b):
    global current_fig
    if current_fig is not None:
        current_fig.write_html('visualizer_output.html', auto_open=False)
        with output:
            print('Saved visualizer_output.html')
    else:
        with output:
            print('No figure to save. Run the simulation first.')

run_button.on_click(on_run_clicked)
save_button.on_click(on_save_clicked)

display(controls, output)


Output()

# How to use
- Adjust the controls and click "Run / Update" to re-simulate and render the interactive figure inline.
- Use the "Save HTML" button to export the current figure to visualizer_output.html.